In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from store.operations import StoringPool
from bidding.operations import create_bidding_app, place_bid, cancel_bid, accept_bid, close_bidding
from trading.operations import create_trading_app, place_trade, cancel_trade, accept_trade, close_trading
from utils import *
from account import Account

In [2]:
dotenv.load_dotenv('.env')

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

In [3]:
creator = Account.from_mnemonic(os.environ.get('CREATOR_MN'))
bidder = Account.from_mnemonic(os.environ.get('BUYER_MN'))
seller = Account.from_mnemonic(os.environ.get('SELLER_MN'))

storeAppID = int(os.environ.get("STORE_APP_ID"))
# close_bidding(client, 693, storeAppID, creator)
print(f"storeAppID: {storeAppID}")

print(f"Creator address: {creator.get_address()}")
print(f"Bidder address: {bidder.get_address()}")
print(f"Seller Address: {seller.get_address()}")

storeAppID: 2185
Creator address: 4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64
Bidder address: X7PMCPPRD2T35ZZRTPETYK53G5YWBKUNONVDY52WODP3N3YM7NGEF5HGWU
Seller Address: Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM


In [4]:
# def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
#     txn = transaction.AssetConfigTxn(
#         sender=sender.get_address(),
#         sp=client.suggested_params(),
#         total=total,
#         decimals=decimals,
#         asset_name=asset_name,
#         unit_name=unit_name,
#         default_frozen=False,
#         strict_empty_address_check=False,
#     )
#     signed_txn = txn.sign(sender.get_private_key())

#     client.send_transaction(signed_txn)

#     response = wait_for_confirmation(client, signed_txn.get_txid())
#     assert response.asset_index is not None and response.asset_index > 0
#     return response.asset_index

# asset_id = create_dummy_asset(seller, 1, 0, "example token", "AVT")

asset_id = int(os.environ.get("TOKEN_ID"))
print(f"The token id is: {asset_id}")

The token id is: 499


In [5]:
start_time = int(time()) + 10
end_time = start_time + 10
price = 1_000_000  # 1 Algo

"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [6]:
storeApp = StoringPool(client=client, creator=creator)
storeApp.create_app()
storeAppID = storeApp.app_id

# storeAppID = int(os.environ.get("STORE_APP_ID"))
# storeApp.app_id = storeAppID
# print(f"Store App ID: {storeAppID}")

storeAppAddress = get_application_address(storeAppID)
print(f"Store App Address: {storeAppAddress}")


Waiting for confirmation...
Transaction YCBKWQD4DAHH6JADRVTSPHOSJW5GPXI4WO7ZR5VJ3TFRPZIQIYCA confirmed in round 112740.
Store App ID: 2479
Store App address: HTYJ7DZKUFHTIHTDXFK4GCXQAEZ33NAQIMKYPKKARY26FD4FQ26LWYT6XI
Waiting for confirmation...
Transaction 23F5RVB6LO6HAPKOPKLVUQOYSX2TQE3NSIICTVFGLUDSCAZVAQZA confirmed in round 112742.
Store App Address: HTYJ7DZKUFHTIHTDXFK4GCXQAEZ33NAQIMKYPKKARY26FD4FQ26LWYT6XI


In [7]:

appID = create_bidding_app(
    client=client,
    creator=creator,
    store_app_id=storeAppID
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction QZ35XPCX24ZNXWQFZ7IUMVANJNPC43Y2LG6L3EP5Y42TKXXG3JKA confirmed in round 112744.
Waiting for confirmation...
Transaction DMJRIAHSAE4BGVM7W6EJ4EHHEXJB2CTXQUSGIXZWQBMHKACAOXBQ confirmed in round 112746.
App ID: 2481
App Address: FFVTUFARWJKGKO6SRPITMPRNSSL2L54JAXHA2IDU5BLYU4LA2Q5RRZOGAM


Alice is setting up and funding token auction...

In [8]:
tradingAppID = create_trading_app(
    client=client,
    creator=creator,
    store_app_id=storeAppID
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction FAZW5OOONMVIF4IRBQFWGV5EPJ6UTOAET7OI4U6MOFMBV5VCMDFQ confirmed in round 112748.
Waiting for confirmation...
Transaction NBJM3DZQ4Z2LAVZZHAFUQVTFQALAQ7FSNH3YUILXY5ERX5V2CNJQ confirmed in round 112750.
App ID: 2481
App Address: FFVTUFARWJKGKO6SRPITMPRNSSL2L54JAXHA2IDU5BLYU4LA2Q5RRZOGAM


In [9]:
storeApp.set_up(trade_app_id=tradingAppID, bid_app_id=appID, auction_app_id=123)

Waiting for confirmation...
Transaction K3AQAQZ2H6NHF2H4CMGHUUMI3OUOA4CT35H2APPNXOMKE7RGZTBQ confirmed in round 112752.


In [10]:
sellerAlgosBefore = get_balances(client, seller.get_address())[0]

print("Alice's algo balance: ", sellerAlgosBefore, " algos")

Alice's algo balance:  20055098667054  algos


In [11]:
sleep(5)
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 100000}


In [12]:
bidPrice = price
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidPrice, " algos")

Carla wants to bid on token, her algo balance:  10025400676091  algos
Carla is placing bid for:  1000000  algos


In [13]:
trading_index = place_trade(client=client, app_id=tradingAppID, seller=seller, token_id=asset_id, token_amount=10, price=bidPrice, trading_index="")

print("Carla bid", trading_index)

store_app_id 2479
seller Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM opt in app 2479
Waiting for confirmation...
Transaction YOOMPU2Y6AJP26JOTK4KDDTRSXUBLGDERX4BY4D2WEPTUU2ODOXA confirmed in round 112755.
Waiting for confirmation...
Transaction 5GHRUH5IMM7EOCDSOAD54TX2K2IUIHX3XG7TZ4XNFGBCBSZ52IKA confirmed in round 112757.
new address: KK2FQ7CUDDGKWRUYROILJ3OIGD52JFUJQV5N3VSZF44J5DLTUUEBVQY7XQ
new private_key: xjIP9BgnNrwx3VWQZJTI8bmp1U3f80X5csY4NDLDssxStFh8VBjMq0aYi5C07cgw+6SWiYV63dZZLziejXOlCA==
new passphrase: flee devote key immense hobby humor trumpet fiction must face tone pair fat install whip video chunk slogan glimpse drum good radio grain absorb easily
Waiting for confirmation...
Transaction PRT3DF4ZASPFBCBDQG3T674FAAHKHVVCF5OKN6KNVDJSGRAZZWOA confirmed in round 112759.
Waiting for confirmation...
Transaction RN7ZTWCWXDYXKLAVAI6DYUDDJRTHLOL5IGKMNXOVJTYBILTKJQJQ confirmed in round 112761.
Waiting for confirmation...
Transaction VHQUZCN27YK7P5FMIV4PDSMZFM5JIIUVA2

In [14]:
accept_trade(client, tradingAppID, bidder, seller.get_address(), trading_index)

token_amount 10
price 1000000
Waiting for confirmation...
Transaction LAYBXRZNUHQIEBJSTL77IM7ZTBEUWJEEBA6M35K62PJOATQKUQOA confirmed in round 112767.


AlgodHTTPError: TransactionPool.Remember: transaction YDAE7EDNNXK3FZXND4V3VDAUUBL72QZGTY5Q54NGK3XQDVLIMZQQ: logic eval error: assert failed pc=468

In [ ]:
bid_index = place_bid(client=client, app_id=appID, token_id=asset_id, bidder=bidder, bid_amount=10, bid_price=bidPrice, bid_index="")

print("Carla bid", asset_id)

In [ ]:
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

In [ ]:
# account_info = client.account_info(bidder.get_address())  
# print(account_info.get('apps-local-state', []))
# print(is_opted_in_app(client, appID, bidder.get_address()))

# cancel_bid(client, appID, bidder, bid_index)

# print("Carla canceled bid", asset_id)

In [ ]:
# bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=10, bid_price=bidPrice, bid_index="")
# print(bid_index)
# bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=20, bid_price=bidPrice+1_000_000, bid_index=bid_index)
# print(bid_index)
# bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=15, bid_price=bidPrice+500_000, bid_index=bid_index)
# print(bid_index)


In [ ]:
bidderAppLocalState = get_app_local_state(client, appID, bid_index)
print(bidderAppLocalState)

In [ ]:
accept_bid(client, appID, seller, bidder.get_address(), bid_index)

Alice is closing out the bidding contract....

In [ ]:
sleep(5)

close_bidding(client, appID, creator, [asset_id])

In [ ]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)

#assert actualAppBalances == expectedAppBalances

In [ ]:
bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]

print("Carla's NFT balance:", bidderNftBalance, " for token ID: ", asset_id)

assert bidderNftBalance == 1

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")

actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidPrice - 1_000
assert actualSellerBalances[asset_id] == 0

In [ ]:
storeAppState = get_app_global_state(client, 1676)
print(storeAppState)
#print(encoding.encode_address(storeAppState[b"C"]))